# ПРОЕКТ Предсказание стоимости жилья

В проекте нам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели используем метрики RMSE, MAE и R2.

**В колонках датасета содержатся следующие данные**

longitude — широта;

latitude — долгота;

housing_median_age — медианный возраст жителей жилого массива;

total_rooms — общее количество комнат в домах жилого массива;

total_bedrooms — общее количество спален в домах жилого массива;

population — количество человек, которые проживают в жилом массиве;

households — количество домовладений в жилом массиве;

median_income — медианный доход жителей жилого массива;

median_house_value — медианная стоимость дома в жилом массиве;

ocean_proximity — близость к океану.

In [1]:
# импортируем необходимые библиотеки

import pandas as pd 

import numpy as np

from pyspark.sql.types import *

import pyspark.sql.functions as F

import matplotlib.pyplot as plt

import seaborn as sns

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler

from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.sql.functions import mean

RANDOM_SEED = 2022


# Подготовка данных

In [2]:
# инициализируем локальную Spark-сессию

import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()


In [3]:
# прочитаем содержимое файла

data = spark.read.load('/datasets/housing.csv', format='csv', sep=",", inferSchema=True, header="true")

In [4]:
# выведем на экран первые пять строк

data.show(5)

# выведим названия колонок 

print(pd.DataFrame(data.dtypes, columns=['column', 'type']))

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [5]:
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [6]:
# выведим базовые статистики

data.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


В датасете 20_640 строк. В жилых массивах Калифорнии проживает преимущественно молодое население - медианный возраст жителей от  1 до 52 лет, со средним медианным возрастом 28 лет. Общее количество комнат находится в диапазоне от 2 до 39 320. По условию задачи, это имеется ввиду общее количество комнат в домах жилого массива,  то же самое касается и количества спален - от 1 до 6445. Количество домовладений в каждом жилом массиве варьируется от 1 до 6082, с медианным значением, равным 500 домовладений на один массив. Средний медианный доход жителей составляет почти 4 дол(от 0.5 дол до 15 дол), а средняя медианная стоимость дома почти 207 тыс дол(от 15 тыс дол до 500 тыс дол). По поводу близости к океану - жилые массивы расположены не более, чем в часе езды до океана или около залива.

## Предобработка данных

Исследуем данные на наличие пропусков и заполним их.

In [7]:
# выведим пропущенные значения в каждой колонке

columns = data.columns

for column in columns:
    print(column, data.filter(data[column].isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Пропущенные значения имеются в одной колонке - количество спальных комнат. Пропущенные значения заменим на среднее.

In [8]:
# пропущенные значения заменяем средним значением

data = data.na.fill(data.select(F.mean(data['total_bedrooms'])).collect()[0][0])

In [10]:
# выведите пропущенные значения в каждой колонке
columns = data.columns

for column in columns:
    print(column, data.filter(data[column].isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


In [11]:
# разделим колонки на два типа: числовые и текстовые, которые представляют категориальные данные

categorical_cols = ["ocean_proximity"]

numerical_cols  = ["housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]

target = "median_house_value" 
    

In [12]:
# разделяем на тренировочную и тестовую выборки

train_data, test_data = data.randomSplit([.8,.2], seed=RANDOM_SEED)

print(train_data.count(), test_data.count()) 

16418 4222


In [13]:
# трансформируем категориальный признак с помощью трансформера StringIndexer (чтобы перевел string в числовое представление)

indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols], handleInvalid="skip") 

data_indexer = indexer.fit(train_data)

train_data = data_indexer.transform(train_data)

test_data = data_indexer.transform(test_data)

train_data.show(3)

test_data.show(3)


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|
|  -124.27|   40.69|              36.0|     2349.0|         528.0|    1194.0|     465.0|       2.5179|           79000.0|     NEAR OCEAN|                2.0|
+---------+--------+------------------+-----------+-

Дополнительно создаем Ohe кодирование с помощью One hot encoding.

In [14]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])

data_encoder = encoder.fit(train_data)

train_data = data_encoder.transform(train_data)

test_data = data_encoder.transform(test_data)

train_data.show(5) 

test_data.show(5) 

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|
|  -124.27|   40.69|              36.0|     2349.0|         528.0|    1194.0|     465.0|       2.5179|        

In [15]:
# объединим признаки в один вектор, с которым ML-алгоритм умеет работать

categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")

test_data = categorical_assembler.transform(test_data) 

train_data = categorical_assembler.transform(train_data) 

train_data.show(5)

test_data.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|
|  -1

In [16]:
# для числовых признаков применим шкалирование значений — чтобы сильные выбросы не смещали предсказания модели

numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                    outputCol="numerical_features")

train_data = numerical_assembler.transform(train_data) 

test_data = numerical_assembler.transform(test_data)

standardScaler = StandardScaler(inputCol='numerical_features',
                                        outputCol="numerical_features_scaled")

standardScaler_data = standardScaler.fit(train_data)

test_data = standardScaler_data.transform(test_data)

train_data = standardScaler_data.transform(train_data)

train_data.show(5)

test_data.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|numerical_features_scaled|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|[52.0,1820.0,300....|     [4.13230773726505...|
|   -124.3|    4

In [17]:
# соберем трансформированные категорийные и числовые признаки с помощью VectorAssembler

all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 

train_data = final_assembler.transform(train_data)

test_data = final_assembler.transform(test_data)


In [18]:
# посмотрим, как выглядит датафрейм после преобразований

train_data.show()

test_data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|numerical_features_scaled|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|[

Переходим к построению моделей.

# Обучение моделей

Для сравнения построим модели на двух разных наборах данных:
 
 * вся выборка (будем использовать признаки features)
 
 * только числовые признаки ( будем использовать numerical_features_scaled).
 

In [19]:
# создадим и обучим модель линейной регрессии на первом наборе данных

lr_1 = LinearRegression(featuresCol='features',labelCol=target)

model_1 = lr_1.fit(train_data)

23/07/16 10:53:05 WARN Instrumentation: [39c62847] regParam is zero, which might cause numerical instability and overfitting.
23/07/16 10:53:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/07/16 10:53:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/07/16 10:53:07 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/07/16 10:53:07 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [20]:
# проверим, как предсказывает модель

predictions_1 = model_1.transform(test_data)

predictedLabes_1 = predictions_1.select("median_house_value", "prediction")

predictedLabes_1.show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|196113.03890509784|
|           50800.0|238373.93600748837|
|           58100.0| 160938.9236801582|
|           68400.0| 176756.2730417095|
|           72200.0|186201.04770909058|
|           67000.0| 175524.8522193165|
|           81300.0|174067.04397184268|
|           70500.0|188199.30296996282|
|           60000.0|164968.63325759236|
|          109400.0|  217901.749742163|
+------------------+------------------+
only showing top 10 rows



In [21]:
# создадим и обучим модель линейной регрессии на втором наборе данных

lr_2 = LinearRegression(featuresCol='numerical_features_scaled',labelCol=target)

model_2 = lr_2.fit(train_data)

23/07/16 10:53:10 WARN Instrumentation: [3f4e29aa] regParam is zero, which might cause numerical instability and overfitting.


In [22]:
# проверим, как предсказывает модель

predictions_2 = model_2.transform(test_data)

predictedLabes_2 = predictions_2.select("median_house_value", "prediction")

predictedLabes_2.show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|144295.34129911353|
|           50800.0|211237.80416117882|
|           58100.0|113264.49794334997|
|           68400.0|126149.45783470222|
|           72200.0|  143898.505418506|
|           67000.0| 131964.0198862102|
|           81300.0|   126850.69839998|
|           70500.0|152048.17904726154|
|           60000.0|131126.90527815287|
|          109400.0| 168111.9848881244|
+------------------+------------------+
only showing top 10 rows



На первый взгляд, второй набор дает лучшие результаты, проверим показатели качества обеих моделей.

# Анализ результатов

Результаты моделей будем оценивать с помощью следующих метрик качества: RMSE (Корень из среднеквадратичной ошибки), MAE (Cредняя абсолютная ошибка ) и R2 (коэффициент детерминации). 

In [23]:
# для анализа результатов работы модели создадим объект класса RegressionEvaluator:

evaluator = RegressionEvaluator(predictionCol='prediction', labelCol = target)

In [24]:
# расчитаем rmse для обеих моделей

print('Показатель RMSE на первой выборке:', evaluator.evaluate(predictions_1, {evaluator.metricName: "rmse"}))


Показатель RMSE на первой выборке: 69320.98513259148


In [25]:

print('Показатель RMSE на второй выборке:', evaluator.evaluate(predictions_2, {evaluator.metricName: "rmse"}))

Показатель RMSE на второй выборке: 74808.82491049665


Чем точнее модель, тем больше значение RMSE будет стремиться к нулю. Таким образом, первая модель показала лучший результат. Но оба показателя имеют очень большие значения, а значит, высока разница между предсказанием и фактическим значением.


In [26]:
# расчитаем mae для обеих моделей

print('Показатель MAE на первой выборке:', evaluator.evaluate(predictions_1, {evaluator.metricName: "mae"}))

Показатель MAE на первой выборке: 50181.25391055931


In [27]:
print('Показатель MAE на второй выборке:', evaluator.evaluate(predictions_2, {evaluator.metricName: "mae"}))

Показатель MAE на второй выборке: 54858.569834715694


Первая выборка снова показывает результат лучше, чем вторая. Но показатель MAE очень далек от нуля, что говорит о низком качестве обеих моделей.

In [28]:
# расчитаем r2 для обеих моделей

print('Показатель R2 на первой выборке:',evaluator.evaluate(predictions_1, {evaluator.metricName: "r2"}))

Показатель R2 на первой выборке: 0.6444159006711733


In [29]:

print('Показатель R2 на второй выборке:',evaluator.evaluate(predictions_2, {evaluator.metricName: "r2"}))

Показатель R2 на второй выборке: 0.5858873030804909


In [ ]:
spark.stop()

И снова показатель на первой выборке лучше. Чем ближе R2 к единице, тем лучше предсказывает модель. В целом, показатели R2 для обеих выборок показывают, что модели предсказывают неплохо.

# Вывод

В проекте мы обучили модель линейной регрессии на данных о жилье в Калифорнии в 1990 году, для этого использовали данные двух выборок: все признаки и только количественные выборки. Модель, для которой использовали первую выборку, показала лучшие результаты метрик качества. Хотя первоначально показалось, что она, наоборот, предсказывает хуже. Таким образом, оставив категориальный признак, а именно, показатель приближенности жилых массивов к океану, мы получили более высокое качество предсказаний.